# Análisis Exploratorio de Datos y Visualización para Consultoría Empresarial
Este cuaderno permite realizar un análisis exploratorio guiado sobre datos organizacionales simulados, con el fin de apoyar procesos de diagnóstico y mejora continua en el contexto de la Gerencia Estratégica de la Calidad.

## 1. Carga y Preprocesamiento de Datos

In [ ]:
# Asistente de carga desde Google Colab
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd
import io

produccion = pd.read_csv(io.BytesIO(uploaded['ProduccionMensual.csv']))
satisfaccion = pd.read_csv(io.BytesIO(uploaded['SatisfacciónClientes.csv']))
auditorias = pd.read_csv(io.BytesIO(uploaded['AuditoríasInternas.csv']))
costos = pd.read_csv(io.BytesIO(uploaded['CostosCalidad.csv']))

### Limpieza de valores nulos, conversión de fechas y revisión de duplicados

In [ ]:
# Conversión de fechas
satisfaccion['Fecha'] = pd.to_datetime(satisfaccion['Fecha'])
auditorias['Fecha'] = pd.to_datetime(auditorias['Fecha'])

# Valores nulos
print('Valores nulos por tabla:')
print('Producción:', produccion.isnull().sum())
print('Satisfacción:', satisfaccion.isnull().sum())
print('Auditorías:', auditorias.isnull().sum())
print('Costos:', costos.isnull().sum())

# Duplicados
print('\nDuplicados por tabla:')
print('Producción:', produccion.duplicated().sum())
print('Satisfacción:', satisfaccion.duplicated().sum())
print('Auditorías:', auditorias.duplicated().sum())
print('Costos:', costos.duplicated().sum())

## 2. Análisis Descriptivo

In [ ]:
# Estadísticos básicos
produccion.describe(), satisfaccion.describe(), costos.describe()

## 3. Detección de Patrones y Correlaciones

In [ ]:
# Correlación entre fallas y tiempo de ciclo
produccion[['Unidades Defectuosas', 'Tiempo de Ciclo']].corr()

In [ ]:
# Relación entre costos de calidad y unidades defectuosas (requiere merge)
df_merge = produccion.merge(costos, on='Mes')
df_merge[['Unidades Defectuosas', 'Costo Fallas Internas', 'Costo Fallas Externas']].corr()

## 4. Visualizaciones

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')

# Series de tiempo
plt.figure(figsize=(10, 5))
plt.plot(produccion['Mes'], produccion['Unidades Defectuosas'], marker='o')
plt.title('Unidades Defectuosas por Mes')
plt.xticks(rotation=45)
plt.ylabel('Defectos')
plt.show()

In [ ]:
# Boxplot por tipo de reclamo
plt.figure(figsize=(8, 5))
sns.boxplot(data=satisfaccion, x='Tipo de Reclamo', y='Satisfacción (1-5)')
plt.title('Satisfacción por Tipo de Reclamo')
plt.show()

In [ ]:
# Mapa de calor de auditorías
pivot_audit = auditorias.pivot_table(index='Área', columns=auditorias['Fecha'].dt.month, values='No Conformidades', aggfunc='sum')
plt.figure(figsize=(10, 6))
sns.heatmap(pivot_audit, annot=True, cmap='YlOrRd')
plt.title('Mapa de Calor de No Conformidades por Área y Mes')
plt.show()

In [ ]:
# Dispersión satisfacción vs tiempo de respuesta
plt.figure(figsize=(8, 5))
sns.scatterplot(data=satisfaccion, x='Tiempo de Respuesta', y='Satisfacción (1-5)', hue='Resuelto')
plt.title('Relación entre Tiempo de Respuesta y Satisfacción')
plt.show()

## 5. Diagnóstico y Conclusiones

In [ ]:
# Áreas con más no conformidades
print(auditorias.groupby('Área')['No Conformidades'].sum().sort_values(ascending=False))

In [ ]:
# Análisis de reclamos
print(satisfaccion['Tipo de Reclamo'].value_counts())

In [ ]:
# Distribución de costos
costos[['Costo Prevención', 'Costo Evaluación', 'Costo Fallas Internas', 'Costo Fallas Externas']].plot(kind='bar', stacked=True, figsize=(12,6))
plt.title('Distribución de Costos de Calidad')
plt.xlabel('Mes')
plt.ylabel('Costo')
plt.xticks(ticks=range(12), labels=costos['Mes'], rotation=45)
plt.show()